In [78]:
!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/data.txt
data = open('data.txt').read()

--2024-04-24 09:10:30--  https://storage.googleapis.com/protonx-cloud-storage/data.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.170.207, 142.251.175.207, 74.125.24.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.170.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘data.txt.5’

data.txt.5          100%[===================>]  91.38K   393KB/s    in 0.2s    

2024-04-24 09:10:30 (393 KB/s) - ‘data.txt.5’ saved [93578/93578]



In [79]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.utils as ku
import numpy as np

In [80]:
data = data.lower().split("\n")
data[:10]

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,']

In [84]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1

import pickle
tokenizer_path = "tokenizer.pickle"
with open(tokenizer_path, "wb") as f:
    pickle.dump(tokenizer, f)

input_sequences = []
for line in data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X_train, y_train = input_sequences[:,:-1],input_sequences[:,-1]
y_train = ku.to_categorical(y_train, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(total_words/2, activation='relu'))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(optimizer = 'adam', loss ='categorical_crossentropy', metrics = ['acc'])
history = model.fit(X_train, y_train, epochs=10, verbose=1)

Epoch 1/10
484/484 [==============================] - 18s 22ms/step - loss: 0.9977 - acc: 0.7529
Epoch 2/10
484/484 [==============================] - 6s 13ms/step - loss: 0.8499 - acc: 0.7854
Epoch 3/10
484/484 [==============================] - 6s 13ms/step - loss: 0.8074 - acc: 0.7952
Epoch 4/10
484/484 [==============================] - 6s 13ms/step - loss: 0.8031 - acc: 0.7961
Epoch 5/10
484/484 [==============================] - 6s 13ms/step - loss: 0.7653 - acc: 0.8005
Epoch 6/10
484/484 [==============================] - 6s 13ms/step - loss: 0.7246 - acc: 0.8126
Epoch 7/10
484/484 [==============================] - 6s 12ms/step - loss: 0.7379 - acc: 0.8077
Epoch 8/10
484/484 [==============================] - 6s 12ms/step - loss: 0.7226 - acc: 0.8115
Epoch 9/10
484/484 [==============================] - 6s 11ms/step - loss: 0.6716 - acc: 0.8223
Epoch 10/10
484/484 [==============================] - 6s 13ms/step - loss: 0.6618 - acc: 0.8265


In [ ]:
model.save('word_generator.keras')

In [ ]:
test_seq = 'they'
next_words = 10

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([test_seq])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = np.argmax(model.predict(token_list))
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  test_seq += " " + output_word
print(test_seq)

1/1 [==============================] - 0s 19ms/step
they are but dressings of a former sight an brow had
